In [1]:
rm(list=ls())
graphics.off()
options(scipen = 999)
library(ggrepel)
library(dplyr)

set.seed(991) #this is the R mitoproteomics seed

# load data
data <- read.csv("raw/mitoproteomics_chronic_data.txt", sep = "\t", header = T)
names(data) <- c("gene", "lfqctrl1", "lfqctrl2", "lfqctrl3", "lfqctrl4", "lfqctrl5", "lfqeae1", "lfqeae2", "lfqeae3", "lfqeae4", "lfqeae5")
data <- data[!is.na(data$gene), ]
data$gene <- gsub(";", "/", data$gene) # replace ";"

data = data[1:10] # exclude c-eae-5 (pca outlier)

# validation: >= 3 replicates per protein
data$validation <- sapply(1:nrow(data), function(x){
  ifelse(length(which(data[x, 2:6] != 0)) >= 3 |
           length(which(data[x, 7:10] != 0)) >= 3,
         1, 0)}
)
data <- data[data$validation == 1, ]
data <- data[, -13]
row.names(data) <- c(1:nrow(data))
data[data == 0] <- NA #replace "0" by "NA"

# log2 transformation
#data[, -c(1, 12)] <- apply(data[, -c(1, 12)], 2, log2) # already transformed

#write.table(data, "mitoproteomics_chronic_output_nopi.txt", sep="\t", row.names=FALSE, quote=FALSE)

# NA imputations
min_value_ctrl = data[, 2:6] %>% min(na.rm=T)
min_value_eae = data[, 7:10] %>% min(na.rm=T)
sd_ctrl = data[, 2:6] %>% unlist() %>% as.vector() %>% sd(na.rm=T)
mean_ctrl = data[, 2:6] %>% unlist() %>% as.vector() %>% mean(na.rm=T)
sd_eae = data[, 7:10] %>% unlist() %>% as.vector() %>% sd(na.rm=T)
mean_eae = data[, 7:10] %>% unlist() %>% as.vector() %>% mean(na.rm=T)
for(i in 1:nrow(data)) {
    if (data[i, 7:10] %>% is.na() %>% sum() == length(data[i, 7:10])) {
        data[i, 7:10] = rnorm(length(data[i, 7:10]), mean=(mean_eae - 2 * sd_eae), sd=(0.3 * sd_eae))
    }
    if (data[i, 2:6] %>% is.na() %>% sum() == length(data[i, 2:6])) {
        data[i, 2:6] = rnorm(length(data[i, 2:6]),  mean=(mean_ctrl - 2 * sd_ctrl), sd=(0.3 * sd_ctrl))
    }
}
#write.table(data, "mitoproteomics_chronic_output_nomice.txt", sep="\t", row.names=FALSE, quote=FALSE)

# mice (doi.org/10.1093/nar/gkaa498, 10.18637/jss.v045.i03)
library(mice)
mice_list = list()
for (n in c('lfqctrl', 'lfqeae')){
    imputation = data[, colnames(data)[grepl(n, colnames(data))]]
    mice = mice(imputation, m = 5, maxit = 50, meth = 'pmm', printFlag = F)
    data[, colnames(data)[grepl(n, colnames(data))]] <- complete(mice, 1)
    mice_list[[n]] <- mice
}

# mean log2lfq and mean log2fc
data$mean_log2lfqctrl <- apply(data[, 2:6], 1, mean, na.rm = T)
data$mean_log2lfqeae <- apply(data[, 7:10], 1, mean, na.rm = T)

data$log2fc <- sapply(1:nrow(data), function(x){
    mean(as.numeric(data[x, 7:10])) - mean(as.numeric(data[x, 2:6]))
})

data$p = sapply(1:nrow(data), function(x){
    t.test(data[x, 7:10], data[x, 2:6])$p.value
})

data$q = p.adjust(data$p %>% as.vector(), method = "BH")

data$neglogq = -log2(data$q)

# significant and regulated candidates
data$enriched <- sapply(1:nrow(data), function(i){
  ifelse(data[i, 'q'] <= 0.05 & xor(data[i, 'log2fc'] <= (-log2(1.5)), data[i, 'log2fc'] >= (log2(1.5))),
         1, 0)
})

# identify mitochondrial proteins (mitocarta)
mitocarta <- as.vector(as.character(read.csv("repos/Mouse.MitoCarta2.0.csv", sep=";")[, 4]))
data$mitocarta <- sapply(1:nrow(data), function(i){ifelse(is.element(data[i, 'gene'], mitocarta), 1, 0)})

write.table(data, "mitoproteomics_chronic_preprocessed.txt", sep="\t", row.names=FALSE, quote=FALSE)

Loading required package: ggplot2


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'mice'


The following objects are masked from 'package:base':

    cbind, rbind


Warning message:
"Number of logged events: 1226"
